# TORCHAUDIO 이용 음성 인식

데이터 세트 가져오기
---------------------

torchaudio를 사용하여 데이터 세트를 다운로드하고 나타냅니다. 여기에서는 서로 다른 사람들이 말한 35 개 명령의 데이터세트 인 SpeechCommands 를 사용 합니다.  SPEECHCOMMANDS는 이 데이터세트의 torch.utils.data.Dataset 버전입니다. 이 데이터세트에서 모든 오디오 파일의 길이는 약 1 초입니다 (따라서 약 16000 time frame 길이 / 16kHz(16,000Hz)).

실제 로딩 및 포맷 단계는 데이터 포인트에 액세스 할 때 발생하며 torchaudio는 오디오 파일을 텐서로 변환합니다. 오디오 파일을 직접로드하려면torchaudio.load()를 사용할 수 있습니다. 오디오 파일의 샘플링 주파수 (SpeechCommands의 경우 16kHz)와 함께 새로 생성된 텐서를 포함하는 튜플을 반환합니다.

데이터 세트로 돌아가서 training, validation, testing 하위 집합으로 분할하는 subclass를 만듭니다.

In [ ]:
# SubsetSC 클래스는 SPEECHCOMMANDS 데이터셋을 상속하여,
# 학습, 테스트, 검증용 하위 집합으로 나누기 위한 클래스
class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        # 부모 클래스(SPEECHCOMMANDS)를 호출하며 데이터셋을 다운로드함
        # 파일에서 리스트를 로드하는 함수
        def load_list(filename):
            # 파일 경로를 설정
            # 파일을 열어 각 줄에 있는 파일 경로를 리스트로 변환
        # 검증용 데이터셋을 로드할 경우
        # 테스트용 데이터셋을 로드할 경우
        # 학습용 데이터셋을 로드할 경우
            # 검증 및 테스트 리스트에서 제외할 항목들을 설정
            # 학습용 데이터는 검증 및 테스트 데이터에 포함되지 않은 파일들로 구성
# 학습과 테스트 데이터를 생성. 이 튜토리얼에서는 검증 데이터를 사용하지 않음
# 학습 데이터셋의 첫 번째 항목을 가져옴 (waveform, 샘플링 레이트, 레이블, 화자 ID, 발화 번호)

SPEECHCOMMANDS 데이터 세트의 데이터 포인트는 파형 (오디오 신호), 샘플 속도, 발화 (레이블), 화자의 ID, 발화 수로 구성된 튜플입니다.

### torchaudio  사용

데이터 세트에서 사용할 수있는 라벨 목록을 찾아 보겠습니다.

오디오 레이블은 사용자가 말하는 명령입니다.

데이터 포맷
-------------------

파형의 경우 분류 능력을 너무 많이 잃지 않고 빠른 처리를 위해 오디오를 다운 샘플링합니다.

In [ ]:
# 새로운 샘플링 속도를 8000Hz로 설정
# 원래 샘플링 속도에서 새로운 샘플링 속도로 오디오를 리샘플링하는 변환 설정
# 파형(waveform)에 리샘플링 변환을 적용하여 변환된 파형 생성
# 리샘플링된 오디오를 새로운 샘플링 속도로 재생

레이블 목록의 색인을 사용하여 각 단어를 인코딩합니다.

In [ ]:
def label_to_index(word):
    # 레이블에서 단어 index를 tensor로 반환
def index_to_label(index):
    # 레이블의 인덱스에 해당하는 단어를 반환
# label --> index --> label 변환

오디오 녹음 및 발화로 구성된 데이터 포인트 목록을 모델에 대한 두 개의 일괄 텐서로 전환하기 위해 데이터 세트를 일괄적으로 반복할 수 있는 PyTorch DataLoader에서 사용하는 데이터 정렬 함수를 구현합니다.

CNN 모델은 주로 입력 형식으로 (batch_size, in_channels, sequence_length)를 요구합니다.
이 경우, permute(0, 2, 1)을 사용하면 시퀀스 차원과 특징 차원(feature_size)을 교환하여 CNN이 요구하는 입력 형식 (batch_size, feature_size, sequence_length)로 맞출 수 있습니다.

In [ ]:
# 세 개의 텐서를 패딩하여 동일한 길이로 맞춘 배치 생성
# batch_first=True를 설정하면 배치 크기가 첫 번째 차원이 됨, padding_value=0. 으로 0을 사용해 패딩
# 패딩된 배치의 크기 출력
# 패딩된 배치의 차원을 변환하여 두 번째 차원과 세 번째 차원을 교환한 크기 출력

- batch shape 은 nn.Conv1d의 입력에 맞추어 $(N, C_{\text{in}}, L)$으로 출력  
- N: batch size, C: channels, L: sequence length

In [ ]:
def pad_sequence(batch):
    # 0으로 패딩하여 배치 내의 모든 텐서를 동일한 길이로 만듭니다.
def collate_fn(batch):
    # 데이터 튜플의 형식:
    # waveform, sample_rate, label, speaker_id, utterance_number
    # 리스트로 데이터를 모으고, 레이블을 인덱스로 인코딩
    # 텐서 리스트를 배치 텐서로 그룹화
# GPU를 사용할 경우 관련 설정
# 학습 데이터 로더 설정
# 테스트 데이터 로더 설정

네트워크 정의
------------------

원시 오디오 데이터를 처리하기 위해 컨벌루션 신경망을 사용.   
일반적으로 더 고급 변환이 오디오 데이터에 적용되지만 CNN을 사용하여 원시 데이터를 정확하게 처리 할 수 있습니다.


In [ ]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
    def forward(self, x):
def count_parameters(model):

가중치 감소가 0.0001로 설정된 Adam 옵티마이저를 사용. 처음에는 0.01의 학습률로 훈련하지만 scheduler20 에포크 이후 훈련 중에 ``scheduler`` 를 사용 하여 0.001로 줄입니다.

In [ ]:
# weight decay: L2 패널티를 적용 (가중치 감쇠)
# 학습률 스케줄러 설정
# step_size 에포크마다 gamma 비율로 학습률을 감소시킴

네트워크 훈련 및 테스트
--------------------------------

이제 훈련 데이터를 모델에 공급하고 backpropagation 및 optimization 단계를 수행하는 training 함수를 정의해 보겠습니다. 훈련을 위해 우리가 사용할 손실은 negative log-likelihood입니다. 그런 다음 각 Epoch 후에 네트워크를 테스트하여 훈련 중에 정확도가 어떻게 달라지는지 확인합니다.

In [ ]:
def train(model, epoch, log_interval):
        # device에서 직접 전체 배치에 변환 및 모델 적용
        # (batch x 1 x n_output)에 대한 negative log-likelihood 손실 계산
        # 학습 통계 출력
        # 진행 상황 바 업데이트
        # 손실 기록

이제 training 함수가 있으므로 네트워크 정확도를 테스트하기 위한 함수를 만들어야합니다. 모델을 eval()모드로 설정한 다음 테스트 데이터세트에서 추론을 실행합니다. eval()을 호출하여 네트워크의 모든 모듈에서 훈련 변수를 false로 설정합니다. 배치 정규화 및 드롭 아웃 레이어와 같은 특정 레이어는 훈련 중에 다르게 작동하므로 이 단계는 올바른 결과를 얻는 데 중요합니다.

In [ ]:
def number_of_correct(pred, target):
    # 정확한 예측의 개수를 셈
def get_likely_index(tensor):
    # 배치 내 각 요소에 대해 가장 가능성 높은 라벨 인덱스를 찾음
def test(model, epoch):
        # device에서 직접 전체 배치에 변환 및 모델 적용
        # 진행 상황 바 업데이트
    # 에포크별 테스트 결과 출력 (정확도)

이제 네트워크를 훈련하고 테스트 할 수 있습니다. 우리는 10  Epoch 동안 네트워크를 훈련시킨 다음, 학습률을 줄이고 10 Epoch 더 훈련할 것입니다. 훈련 중 정확도가 어떻게 달라지는지 확인하기 위해 각 Epoch 후에 네트워크를 테스트합니다.

In [ ]:
# tqdm을 사용하여 진행 상황 표시
# 학습 손실을 반복 횟수에 따라 시각화

네트워크는 2 Epoch 후에 테스트 세트에서 65 % 이상, 21 Epoch 후에 85 % 이상 정확해야합니다. train 세트의 마지막 단어를보고 모델이 어떻게했는지 살펴 보겠습니다.

In [ ]:
def predict(tensor):
    # 모델을 사용하여 파형의 레이블을 예측
# 학습 세트에서 마지막 항목의 파형(waveform), 샘플링 속도, 발화(utterance) 가져오기
# 오디오 재생 (파형을 numpy 배열로 변환하고 샘플링 속도 설정)
# 예측된 결과와 실제 발화 출력

올바르게 분류되지 않은 예를 찾아 보겠습니다.

In [ ]:
    # 파형을 예측하여 출력값을 생성
    # 예측값이 실제 발화(utterance)와 다를 경우
        # 오디오 재생 (파형을 numpy 배열로 변환하고 샘플링 속도를 적용하여 재생)
        # 잘못 분류된 데이터 포인트와 예측 결과 출력
        # 모든 예시가 올바르게 분류된 경우 출력
        # 잘못된 예시가 없을 경우 마지막 데이터 포인트를 출력
        # 마지막 데이터 포인트의 오디오 재생
        # 마지막 데이터 포인트의 예상 발화와 예측된 결과 출력